# **Simulation in closed-loop**

In [321]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors

import package_LAB
from importlib import reload
package_LAB = reload(package_LAB)

from package_LAB import LL_RT, PID_RT, IMCTuning
from package_DBR import SelectPath_RT, Delay_RT, FO_RT


In [322]:
#plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interactive, VBox, IntRangeSlider, IntSlider, Checkbox, interactive_output

## Simulation parameters

In [323]:
TSim = 2000
Ts = 1    
N = int(TSim/Ts) + 1 

## Dynamics and controller parameters

In [324]:
#DV 
Kp_ODV_SOPDT = 0.2951290424136788
T1_ODV_SOPDT = 182.2549613489765
T2_ODV_SOPDT = 13.184430234847984
theta_ODV_SOPDT = 28.999891911961512

#MV
Kp_OMV_SOPDT = 0.30788564834253684
T1_OMV_SOPDT = 183.81942938046797
T2_OMV_SOPDT = 3.2920224028341535e-12
theta_OMV_SOPDT = 20.015407110302775

#Operating points 
DV0 = 50 
MV0 = 50
PV0 = 49.3

# Set maximum and minimum MV values
MVmin = 0
MVmax = 100

# Coefficients
alpha = 0.7
gamma = 0.5

## Scenarios

In [325]:
scenario = 6

# 1 : Response to setpoint change
if scenario == 1:
    SPPath = {0: PV0, 1000: PV0 + 10}
    ManPath = {0: True, 500: False}
    MVManPath = {0: MV0}
    DVPath = {0: DV0}
    FF = False
    ManFF = True
# 2 : Response to perturbation, with controller in manual mode, without FF
elif scenario == 2:
    SPPath = {0: PV0}
    ManPath = {0: True}
    MVManPath = {0: MV0}
    DVPath = {0: DV0, 1500: DV0 + 10}
    FF = False
    ManFF = True
# 3 : Response to perturbation, with controller in manual mode, with FF
elif scenario == 3:
    SPPath = {0: PV0}
    ManPath = {0: True}
    MVManPath = {0: MV0}
    DVPath = {0: DV0, 1500: DV0 + 10}
    FF = True
    ManFF = True
# 4 : Response to perturbation, with controller in auto mode, without FF
elif scenario == 4:
    SPPath = {0: PV0}
    ManPath = {0: False}
    MVManPath = {0: MV0}
    DVPath = {0: DV0, 1500: DV0 + 10}
    FF = False
    ManFF = False # Not needed
# 5 : Response to perturbation, with controller in auto mode, with FF
elif scenario == 5:
    SPPath = {0: PV0}
    ManPath = {0: False}
    MVManPath = {0: MV0}
    DVPath = {0: DV0, 1500: DV0 + 10}
    FF = True
    ManFF = False # Not needed
# 6 : Response to perturbation & setpoint change, with controller in auto mode, with FF
elif scenario == 6:
    SPPath = {0: PV0, 1000: PV0 + 10}
    ManPath = {0: False}
    MVManPath = {0: MV0}
    DVPath = {0: DV0, 1500: DV0 + 10}
    FF = True
    ManFF = False # Not needed

## Running simulation

In [326]:
# IMC tuning
Kc, TI, TD = IMCTuning(Kp_OMV_SOPDT, T1_OMV_SOPDT, T2_OMV_SOPDT, theta_OMV_SOPDT, gamma, model="SOPDT")
print(f"Kc: {Kc}, Ti: {TI}, Td: {TD}")

Kc: 5.33426261068635, Ti: 183.81942938047126, Td: 3.2920224028340946e-12


### Initialization of arrays and parameters

In [327]:
# Running simulation with chosen scenario

t = []
FF = True

SP = []
PV = []
MAN = []
MV_MAN = []
DV = []
MVFF = []
MV = []
MVp = []
MVi = []
MVd = []
E = []
PV_p = []
PV_d = []

MVFF_Delay = []
MVFF_LL1 = []
MV_Delay_P = []
MV_FO_P = []
MV_Delay_D = []
MV_FO_D = []



for i in range(0, N):
    t.append(i * Ts)
    SelectPath_RT(SPPath, t, SP)
    SelectPath_RT(ManPath, t, MAN)
    SelectPath_RT(MVManPath, t, MV_MAN)
    SelectPath_RT(DVPath, t, DV)
    
    # FeedForward
    Delay_RT(DV - DV0*np.ones_like(DV), max(theta_ODV_SOPDT-theta_OMV_SOPDT, 0), Ts, MVFF_Delay)
    LL_RT(MVFF_Delay, -Kp_ODV_SOPDT/Kp_OMV_SOPDT, T1_OMV_SOPDT, T1_ODV_SOPDT, Ts, MVFF_LL1)
    if FF == True:
        LL_RT(MVFF_LL1, 1, T2_OMV_SOPDT, T2_ODV_SOPDT, Ts, MVFF)
    else:
        LL_RT(MVFF_LL1, 0, T2_OMV_SOPDT, T2_ODV_SOPDT, Ts, MVFF) # Set MVFF to 0 if FF is disabled
    
    # PID
    PID_RT(SP, PV, MAN, MV_MAN, MVFF, Kc, TI, TD, alpha, Ts, MVmin, MVmax, MV, MVp, MVi, MVd, E, ManFF, PV0)
    
    # Process
    Delay_RT(MV, theta_OMV_SOPDT, Ts, MV_Delay_P, MV0)
    FO_RT(MV_Delay_P, Kp_OMV_SOPDT, T1_OMV_SOPDT, Ts, MV_FO_P)
    FO_RT(MV_FO_P, 1, T2_OMV_SOPDT, Ts, PV_p)
    
    # Disturbance
    Delay_RT(DV - DV0*np.ones_like(DV), theta_ODV_SOPDT, Ts, MV_Delay_D)
    FO_RT(MV_Delay_D, Kp_ODV_SOPDT, T1_ODV_SOPDT, Ts, MV_FO_D)
    FO_RT(MV_FO_D, 1, T2_ODV_SOPDT, Ts, PV_d)
    
    PV.append(PV_p[-1] + PV_d[-1] + PV0 - Kp_OMV_SOPDT*MV0)

### Plot code

In [328]:
# Create figure
fig = go.FigureWidget(make_subplots(rows=4, cols=1, specs = [[{}], [{}], [{}], [{}]], vertical_spacing = 0.15, row_heights=[0.1, 0.4, 0.4, 0.1], subplot_titles=("Manual Mode", "MV and Components", "PV, SP and E", "Perturbation DV")))
fig.add_trace(go.Scatter(x=t, y=SP, name="SP"), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=PV, name="PV"), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=E, name="E", line=dict(dash='dash')), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=MV, name="MV"), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MVp, name="MVp", line=dict(dash='dash')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MVi, name="MVi", line=dict(dash='dash')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MVd, name="MVd", line=dict(dash='dash')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MAN, name="Man"), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=MV_MAN, name="MVMan"), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=DV, name="DV"), row=4, col=1)

# Update layout
fig['layout'].update(height=800, width=800)
fig['layout']['xaxis1'].update(title='Time (s)')
fig['layout']['yaxis1'].update(title='(°C)')
fig['layout']['xaxis2'].update(title='Time (s)')
fig['layout']['yaxis2'].update(title='MV (%)')
fig['layout']['xaxis3'].update(title='Time (s)')
fig['layout']['xaxis4'].update(title='Time (s)')

def UpdatePlot():
    # Running simulation with chosen scenario
    t = []

    SP = []
    PV = []
    MAN = []
    MV_MAN = []
    DV = []
    MVFF = []
    MV = []
    MVp = []
    MVi = []
    MVd = []
    E = []
    PV_p = []
    PV_d = []

    MVFF_Delay = []
    MVFF_LL1 = []
    MV_Delay_P = []
    MV_FO_P = []
    MV_Delay_D = []
    MV_FO_D = []

    for i in range(0, N):
        t.append(i * Ts)
        SelectPath_RT(SPPath, t, SP)
        SelectPath_RT(ManPath, t, MAN)
        SelectPath_RT(MVManPath, t, MV_MAN)
        SelectPath_RT(DVPath, t, DV)
        
        # FeedForward
        Delay_RT(DV - DV0*np.ones_like(DV), max(theta_ODV_SOPDT-theta_OMV_SOPDT, 0), Ts, MVFF_Delay)
        LL_RT(MVFF_Delay, -Kp_ODV_SOPDT/Kp_OMV_SOPDT, T1_OMV_SOPDT, T1_ODV_SOPDT, Ts, MVFF_LL1)
        if FF == True:
            LL_RT(MVFF_LL1, 1, T2_OMV_SOPDT, T2_ODV_SOPDT, Ts, MVFF)
        else:
            LL_RT(MVFF_LL1, 0, T2_OMV_SOPDT, T2_ODV_SOPDT, Ts, MVFF) # Set MVFF to 0 if FF is disabled
        
        # PID
        PID_RT(SP, PV, MAN, MV_MAN, MVFF, Kc, TI, TD, alpha, Ts, MVmin, MVmax, MV, MVp, MVi, MVd, E, ManFF, PV0)
        
        # Process
        Delay_RT(MV, theta_OMV_SOPDT, Ts, MV_Delay_P, MV0)
        FO_RT(MV_Delay_P, Kp_OMV_SOPDT, T1_OMV_SOPDT, Ts, MV_FO_P)
        FO_RT(MV_FO_P, 1, T2_OMV_SOPDT, Ts, PV_p)
        
        # Disturbance
        Delay_RT(DV - DV0*np.ones_like(DV), theta_ODV_SOPDT, Ts, MV_Delay_D)
        FO_RT(MV_Delay_D, Kp_ODV_SOPDT, T1_ODV_SOPDT, Ts, MV_FO_D)
        FO_RT(MV_FO_D, 1, T2_ODV_SOPDT, Ts, PV_d)
        
        PV.append(PV_p[-1] + PV_d[-1] + PV0 - Kp_OMV_SOPDT*MV0)
        
    fig.data[0].y = SP 
    fig.data[1].y = PV
    fig.data[2].y = E
    fig.data[3].y = MV
    fig.data[4].y = MVp
    fig.data[5].y = MVi
    fig.data[6].y = MVd
    fig.data[7].y = MAN
    fig.data[8].y = MV_MAN
    fig.data[9].y = DV
    
    
    

def Update_Gamma(gammaP):
    global Kc
    global TI
    global TD
    global gamma 
    
    gamma = gammaP
    
    Kc, TI, TD = IMCTuning(Kp_OMV_SOPDT, T1_OMV_SOPDT, T2_OMV_SOPDT, theta_OMV_SOPDT, gamma, model='FOPDT')
    print(f"Kc: {Kc}, Ti: {TI}, Td: {TD}")
    UpdatePlot()

def Update_Alpha(alphaP):
    global alpha
    
    alpha = alphaP
    UpdatePlot()
    
def Update_Manual(manual_time, MVManual, ActiveManual):
    global ManPath
    global MVmanPath
    global ManFF
     
    if ActiveManual:
        ManPath = {0: False, manual_time[0]: True, manual_time[1]: False}
        MVmanPath = {0: 0, manual_time[0]: MVManual, manual_time[1]: 0}
        ManFF = True; 
    else :
        ManFF = False
        ManPath = {0: False}
        MVmanPath = {0: 0}
    
    t = []
    for i in range(1, N):
        t.append(i*Ts)
        SelectPath_RT(ManPath,t,MAN)
        SelectPath_RT(MVmanPath,t,MV_MAN) 
        
    UpdatePlot()
        

def Update_FF(ActiveFF):
    global FF
    FF = ActiveFF
    UpdatePlot()
    
def Update_Perturbation(perturbation, time_perturbation):
    global DVPath
    
    DVPath = {0:perturbation[0], time_perturbation: perturbation[1]}
    UpdatePlot()
    
def Update_SetPoint(setpoint, time_setpoint):
    global SPPath
    
    SPPath = {0:setpoint[0], time_setpoint: setpoint[1]}
    UpdatePlot()

slider_style = {'description_width': 'initial'}
manual_time_slider = IntRangeSlider(min=0, max=1999, step=1, value=[50, 100], description="Manual Time", layout={'width': '500px'}, slider_style=slider_style)
MVManual_slider = IntSlider(min=0, max=100, step=1, value=50, description="MV Manual Value", layout={'width': '500px'}, slider_style=slider_style)
ActiveManual_checkbox = Checkbox(value=False, description="Active Manual")
PerturbationSlider = IntRangeSlider(min=0, max=100, step=1, value=[50, 100], description="Perturbation Value", layout={'width': '500px'}, slider_style=slider_style)
time_perturbation = IntSlider(min=0, max=1999, step=10, value=50, description="Perturbation Time", layout={'width': '500px'}, slider_style=slider_style)
SetPointSlider = IntRangeSlider(min=0, max=100, step=1, value=[50, 100], description="SetPoint Value", layout={'width': '500px'}, slider_style=slider_style)
time_setpoint = IntSlider(min=0, max=1999, step=10, value=50, description="SetPoint Time", layout={'width': '500px'}, slider_style=slider_style)

        
# Create interactive widget
GammaSlider = interactive(Update_Gamma, gammaP=(0.2, 0.9, 0.02), description='Gamma')
AlphaSlider = interactive(Update_Alpha, alphaP=(0, 0.9, 0.01), description='Alpha')
Manual = interactive(Update_Manual, manual_time=manual_time_slider, MVManual=MVManual_slider, ActiveManual=ActiveManual_checkbox)
FFCheckBox = interactive(Update_FF, ActiveFF=False, description='FeedForward')
Perturbation = interactive(Update_Perturbation, perturbation=PerturbationSlider, time_perturbation=time_perturbation, description='Perturbation')
SetPoint = interactive(Update_SetPoint, setpoint=SetPointSlider, time_setpoint=time_setpoint, description='SetPoint')
    
# Display figure and widget
vb = VBox([fig, Manual, GammaSlider, AlphaSlider, FFCheckBox, Perturbation, SetPoint])
vb.layout.align_items = 'center'
vb  


        

    'data': [{'name': 'SP',
              'type': 'scatter',
              'uid'…